## Imports

To run this experiment, one needs the Deep Learning workspace, which contains tensorflow pre-installed

In [12]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping

import sklearn
import mlflow

## Data

In [13]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [14]:
y_cat_test = to_categorical(y_test,10)

In [15]:
y_cat_train = to_categorical(y_train,10)

In [16]:
x_train = x_train/255
x_test = x_test/255

Reshape to include channel dimension (in this case, 1 channel)

In [17]:
x_train = x_train.reshape(60000, 28, 28, 1)

In [18]:
x_test = x_test.reshape(10000,28,28,1)

# Training the Model

### Model

In [19]:
model = Sequential()

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu',))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (YOU CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # we can add in additional metrics https://keras.io/metrics/

/opt/conda/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

## Train the Model

In [21]:
mlflow.set_experiment('MNIST with TensorFlow 2')
mlflow.keras.autolog()

2024/12/11 22:54:36 INFO mlflow.tracking.fluent: Experiment with name 'MNIST with TensorFlow 2' does not exist. Creating a new experiment.
2024/12/11 22:54:36 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of keras. If you encounter errors during autologging, try upgrading / downgrading keras to a supported version, or try upgrading MLflow.


In [22]:
model.fit(x_train,y_cat_train,epochs=4,validation_data=(x_test,y_cat_test),callbacks=[early_stop])

2024/12/11 22:54:38 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'dedf2084d4254f2a814e9320f9d09185', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current keras workflow
2024-12-11 22:54:43.264342: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/4
1874/1875 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9148 - loss: 0.2937

2024-12-11 22:55:10.829206: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31360000 exceeds 10% of free system memory.


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 14ms/step - accuracy: 0.9148 - loss: 0.2936 - val_accuracy: 0.9815 - val_loss: 0.0546
Epoch 2/4
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 33s 18ms/step - accuracy: 0.9839 - loss: 0.0521 - val_accuracy: 0.9845 - val_loss: 0.0446
Epoch 3/4
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 34s 18ms/step - accuracy: 0.9910 - loss: 0.0304 - val_accuracy: 0.9856 - val_loss: 0.0442
Epoch 4/4
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 27s 15ms/step - accuracy: 0.9939 - loss: 0.0194 - val_accuracy: 0.9868 - val_loss: 0.0401
